In [ ]:
pip install datasets

Note: you may need to restart the kernel to use updated packages.


In [ ]:
!pip install transformers==4.45.2 sentence-transformers==3.1.1

In [ ]:
from datasets import load_dataset
from sentence_transformers import (
    SentenceTransformer,
    SentenceTransformerTrainer,
)
from sentence_transformers.losses import MultipleNegativesRankingLoss
from sentence_transformers.evaluation import TripletEvaluator


c:\Users\Raviksh\miniconda3\envs\chat_arena\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# for my custom dataset

import pandas as pd

train = pd.read_parquet('./train.parquet')

In [ ]:
import pandas as pd

# Initialize an empty list to store rows for the DataFrame
data = []

# Loop through the dataset to create triplets
n_examples = len(train)
for i in range(n_examples):
    prompt = train.iloc[i]['prompt']
    response_a = train.iloc[i]['response_a']
    response_b = train.iloc[i]['response_b']
    label = train.iloc[i]['winner']

    if label == 'model_a':
        data.append({
            "anchor": prompt,
            "positive": response_a,
            "negative": response_b
        })
    elif label == 'model_b':
        data.append({
            "anchor": prompt,
            "positive": response_b,
            "negative": response_a
        })
    else:
        print(f"Error in row {i}: Unexpected label value '{label}'")

# Convert the list of dictionaries into a DataFrame
triplets_df = pd.DataFrame(data)

# Display the first few rows of the DataFrame
print(triplets_df.head())


                                              anchor  \
0                                 vieš po Slovensky?   
1  You will be given a piece of news. Analyze it ...   
2  Dört basamaklı, rakamları birbirinden ve sıfır...   
3  현재 추천된 탑 3 종목인 Cabaletta Bio (CABA), Rocket Ph...   
4                                  Please be boring    

                                            positive  \
0   Áno, hovorím po slovensky. Ako vám môžem pomôcť?   
1  Let's break down the news and analyze it accor...   
2  Bu soruyu çözmek için, verilen koşulları adım ...   
3  현재 추천된 탑 3 종목에 순위를 매기기 위해서는 여러 가지 요소들을 고려해야 합니...   
4  Alright, I'll be as boring as possible.\n\nTod...   

                                            negative  
0  Áno, veď som tu! Môžem ti pomôcť s otázkami al...  
1  ```json\n{\n  "contains_orgs": true,\n  "orgs"...  
2  Bu problemi adım adım çözelim:\n\n1) ABCD - DC...  
3  죄송하지만 저는 금융 조언을 제공할 수 없습니다. 저는 AI 모델이며, 투자 결정에...  
4  Understood. Here is a straightforward, unadorn..

In [ ]:
from sklearn.model_selection import train_test_split

# Split the DataFrame into train and test sets
train_df, test_df = train_test_split(triplets_df, test_size=0.2, random_state=42)

# Display the first few rows of each split
print("Train Set:")
print(train_df.head())

print("\nTest Set:")
print(test_df.head())


Train Set:
                                                  anchor  \
10277  Can you group these words in groups of 4: HAT ...   
1696   よほど楽しいことをしてない限り怒りがこみあげてくるわけよ。もう、ずっと、ものすごく楽しいこと...   
7416   rewrite: \nFuel your tech adventures with the ...   
11168  do a summary of "STATISTICAL PARADISES AND PAR...   
6865   Viết 1 đoạn văn giới thiệu về thành phố Long X...   

                                                positive  \
10277  Here are the grouped words in categories of 4:...   
1696   それは、かなり大変な状況ですね。楽しいことを見つけ続けるのはエネルギーが必要ですし、常に楽し...   
7416   **Elevate Your iPhone 16 Experience with Unmat...   
11168  ## Statistical Paradises and Paradoxes in Big ...   
6865   Thành phố Long Xuyên, thủ phủ của tỉnh An Gian...   

                                                negative  
10277  Here's one possible grouping of the words into...  
1696   心境に感謝し、自分自身の気持ちを正確に捉えることが大切です。日々の快乐を求めるのは当然のこと...  
7416   Embark on your technological journey with the ...  
11168  ##  Statistical Paradise

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("C:\\Users\\Raviksh\\Downloads\\finetune_sentence_transformer\\BAAI_bge_large_en_v1.5")

In [ ]:
loss = MultipleNegativesRankingLoss(model)


In [ ]:
from datasets import Dataset
tr = Dataset.from_dict({
    "anchor": list(train_df['anchor']),
    "positive": list(train_df['positive']),
    "negative": list(train_df['negative'])
})

te = Dataset.from_dict({
    "anchor": list(test_df['anchor']),
    "positive": list(test_df['positive']),
    "negative": list(test_df['negative'])
})

In [ ]:
train_dataset = tr  # Your training dataset
eval_dataset = te  # Your evaluation dataset


In [ ]:
from sentence_transformers import SentenceTransformer, InputExample, losses, SentenceTransformerTrainer, SentenceTransformerTrainingArguments
from torch.optim import AdamW
#from transformers import get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(), lr=6e-5)  # Adjust the learning rate as needed


In [ ]:
# num_training_steps = len(train_dataset) * 5  # Assuming 5 epochs
# num_warmup_steps = int(0.1 * num_training_steps)  # Warmup steps (10% of total steps)
# scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)


In [ ]:
from transformers import get_cosine_schedule_with_warmup
batch_size = 8
num_epochs = 5
num_training_steps = (len(train_dataset) // batch_size) * num_epochs  # Total steps
num_warmup_steps = int(0.1 * num_training_steps)  # 10% warm-up (adjustable)

scheduler = get_cosine_schedule_with_warmup(
    optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=num_training_steps
)


In [ ]:
training_args = SentenceTransformerTrainingArguments(
    output_dir="./results",               # Directory to save results
    num_train_epochs=5,                   # Number of epochs
    per_device_train_batch_size=8,        # Batch size for training
    logging_dir="./logs",                 # Directory for storing logs
    evaluation_strategy="epoch",          # Evaluate at the end of each epoch
    save_strategy="epoch",                # Save model at the end of each epoch
    save_total_limit=5,                   # Keep only 2 checkpoints
    metric_for_best_model="eval_loss",    # Use eval loss to select the best model
    greater_is_better=False,              # Lower eval loss is better
    load_best_model_at_end=True,          # Load the best model after training
)


c:\Users\Raviksh\miniconda3\envs\chat_arena\lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = SentenceTransformerTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss=loss,
    optimizers=(optimizer, scheduler)  # Pass the custom optimizer and scheduler
)


In [ ]:
trainer.train()


  2%|▏         | 500/24220 [25:35<20:01:53,  3.04s/it]

{'loss': 0.9917, 'grad_norm': 14.937872886657715, 'learning_rate': 1.2391573729863692e-05, 'epoch': 0.1}


  4%|▍         | 1000/24220 [51:27<19:57:41,  3.09s/it]

{'loss': 0.844, 'grad_norm': 8.561187744140625, 'learning_rate': 2.4783147459727385e-05, 'epoch': 0.21}


  6%|▌         | 1500/24220 [1:17:16<19:32:44,  3.10s/it]

...

                                                        
100%|██████████| 24220/24220 [22:16:18<00:00,  3.91s/it]

{'eval_loss': 1.0698752403259277, 'eval_runtime': 846.1707, 'eval_samples_per_second': 11.449, 'eval_steps_per_second': 1.431, 'epoch': 5.0}


100%|██████████| 24220/24220 [22:16:44<00:00,  3.31s/it]

{'train_runtime': 80204.4935, 'train_samples_per_second': 2.416, 'train_steps_per_second': 0.302, 'train_loss': 0.6424064959109469, 'epoch': 5.0}


TrainOutput(global_step=24220, training_loss=0.6424064959109469, metrics={'train_runtime': 80204.4935, 'train_samples_per_second': 2.416, 'train_steps_per_second': 0.302, 'total_flos': 0.0, 'train_loss': 0.6424064959109469, 'epoch': 5.0})

# best model i got using 19536 checkpoint i will retrain it for 3 epoch

In [ ]:
from sentence_transformers import SentenceTransformer
checkpoint_path = "C:\\Users\\Raviksh\\Downloads\\finetune_sentence_transformer\\s_tr\\checkpoint-19376"
model = SentenceTransformer(checkpoint_path)

In [ ]:
loss = MultipleNegativesRankingLoss(model)


In [ ]:
from transformers import get_cosine_schedule_with_warmup
batch_size = 8
num_epochs = 3
num_training_steps = (len(train_dataset) // batch_size) * num_epochs  # Total steps
num_warmup_steps = int(0.15 * num_training_steps)  # 10% warm-up (adjustable)


In [ ]:
from sentence_transformers import SentenceTransformer, InputExample, losses, SentenceTransformerTrainer, SentenceTransformerTrainingArguments
from torch.optim import AdamW

In [ ]:


# Load optimizer and scheduler states if saved
optimizer = AdamW(model.parameters(), lr=2e-5)
scheduler = get_cosine_schedule_with_warmup(
    optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=num_training_steps
)

# Update training arguments for retraining
training_args = SentenceTransformerTrainingArguments(
    output_dir="./results",
    num_train_epochs=3,                   # Number of additional epochs
    per_device_train_batch_size=8,
    logging_dir="./logs",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=3,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    load_best_model_at_end=True
)

# Resume training
trainer = SentenceTransformerTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss=loss,
    optimizers=(optimizer, scheduler)  # Use saved or reinitialized optimizer/scheduler
)

trainer.train()


c:\Users\Raviksh\miniconda3\envs\chat_arena\lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  3%|▎         | 500/14532 [27:31<12:24:36,  3.18s/it]

{'loss': 0.2896, 'grad_norm': 9.947051048278809, 'learning_rate': 4.589261128958238e-06, 'epoch': 0.1}


  7%|▋         | 1000/14532 [54:31<12:11:17,  3.24s/it]

{'loss': 0.2533, 'grad_norm': 6.576684474945068, 'learning_rate': 9.178522257916476e-06, 'epoch': 0.21}


 10%|█         | 1500/14532 [1:21:21<11:42:14,  3.23s/it]

...

                                                        
100%|██████████| 14532/14532 [13:17:42<00:00,  3.10s/it]

{'eval_loss': 1.2383840084075928, 'eval_runtime': 195.9146, 'eval_samples_per_second': 49.45, 'eval_steps_per_second': 6.181, 'epoch': 3.0}


100%|██████████| 14532/14532 [13:17:50<00:00,  3.29s/it]

{'train_runtime': 47870.8569, 'train_samples_per_second': 2.428, 'train_steps_per_second': 0.304, 'train_loss': 0.1430878928466963, 'epoch': 3.0}


TrainOutput(global_step=14532, training_loss=0.1430878928466963, metrics={'train_runtime': 47870.8569, 'train_samples_per_second': 2.428, 'train_steps_per_second': 0.304, 'total_flos': 0.0, 'train_loss': 0.1430878928466963, 'epoch': 3.0})

In [ ]:
from sentence_transformers import SentenceTransformer
from huggingface_hub import HfApi, Repository

# Path to the saved model directory
model_dir = "/kaggle/working/results/checkpoint-7500/"

# Load the trained SentenceTransformer model
model = SentenceTransformer(model_dir)

# Push the model to Hugging Face
model.push_to_hub("Raviksh/finetune_bge_large",token='###########')

print(f"Model uploaded to Hugging Face: https://huggingface.co/your-org-name/your-model-name")


model.safetensors:   0%|          | 0.00/711M [00:00<?, ?B/s]

Model uploaded to Hugging Face: https://huggingface.co/your-org-name/your-model-name


In [ ]:
import os

print(os.listdir('./results'))

['checkpoint-7500', 'checkpoint-8000']


In [ ]:
print(os.listdir('./results/checkpoint-7500'))


['README.md', 'scheduler.pt', 'model.safetensors', 'training_args.bin', 'optimizer.pt', 'modules.json', 'tokenizer_config.json', '1_Pooling', 'config.json', 'special_tokens_map.json', 'trainer_state.json', 'tokenizer.json', 'rng_state.pth', 'config_sentence_transformers.json', 'sentence_bert_config.json', 'vocab.txt']
